In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
body_parts = [
            'mouth', 'eye', 'skull', 'upper tail bone', 'lower tail bone',
            'upper tail', 'lower tail', 'pectoral fin', 'anal fin start',
            'anal fin mid', 'dorsal fin_base', 'dorsal fin_tip', 'stomach', 'middle'
        ]

In [ ]:
def prepare_individuals(data_numeric, target_length=141, body_parts=None, num_individuals=8):
    # Define body parts if not provided
    if body_parts is None:
        body_parts = [
            'mouth', 'eye', 'skull', 'upper tail bone', 'lower tail bone',
            'upper tail', 'lower tail', 'pectoral fin', 'anal fin start',
            'anal fin mid', 'dorsal fin_base', 'dorsal fin_tip', 'stomach', 'middle'
        ]

    def process_column(column, target_length):
        result_array = np.zeros(target_length)
        non_nan_indices = np.where(~column.isna())[0]
        if len(non_nan_indices) > 1:
            valid_values = column[non_nan_indices]
            differences = np.diff(valid_values)
            for i, diff in enumerate(differences):
                result_array[non_nan_indices[i + 1]] = diff
        return result_array

    # Dictionary to keep DataFrames for each individual
    individual_features = {}

    for individual in range(1, num_individuals + 1):
        features_list = []  # List to accumulate the features of this individual

        for idx, body_part in enumerate(body_parts):
            # Handle the first individual and the first body part
            if individual == 1 and idx == 0:
                x_col_name = 'x'
                y_col_name = 'y'
            else:
                x_col_name = f'x.{(individual - 1) * len(body_parts) + idx}'
                y_col_name = f'y.{(individual - 1) * len(body_parts) + idx}'

            if x_col_name in data_numeric.columns and y_col_name in data_numeric.columns:
                delta_x = process_column(data_numeric[x_col_name], target_length)
                delta_y = process_column(data_numeric[y_col_name], target_length)

                if len(delta_x) > 0 and len(delta_y) > 0:
                    speed = np.insert(np.sqrt(delta_x**2 + delta_y**2), 0, 0)
                    direction = np.insert(np.arctan2(delta_y, delta_x), 0, 0)
                    direction_degrees = np.degrees(direction)

                    # Add the features to the list
                    features_list.append(speed)
                    features_list.append(direction_degrees)

        # Create a DataFrame for this individual from the list of features
        if features_list:
            individual_features[f'individual{individual}'] = pd.DataFrame(features_list).transpose()

    return individual_features


In [ ]:
data_numeric = pd.read_csv('TrainDataFishVideo.csv', skiprows=3)
print(data_numeric.shape)

train_data = prepare_individuals(data_numeric, target_length=141, body_parts=body_parts, num_individuals=8)
print(train_data.keys())
if 'individual8' in train_data:
    del train_data['individual8']
for key in train_data.keys():
    print(f"{key}: {len(train_data[key].columns)} columns")

(141, 227)
dict_keys(['individual1', 'individual2', 'individual3', 'individual4', 'individual5', 'individual6', 'individual7', 'individual8'])
individual1: 28 columns
individual2: 28 columns
individual3: 28 columns
individual4: 28 columns
individual5: 28 columns
individual6: 28 columns
individual7: 28 columns


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

# Asignar las etiquetas a los individuos
jaime_labels = [0, 0, 2, 2, 1, 2, 1]
jaime_labels = [0, 0, 2, 2, 1, 2, 1]

# Preparar datos de entrenamiento y validación
train_data = []
train_labels = []
validation_data = []
validation_labels = []

for i, (key, df) in enumerate(train_data.items()):
    # Convertir el DataFrame a un array 3D (samples, time steps, features)
    individual_data = np.expand_dims(df.values, axis=0)

    if i < 6:  # Primeros 6 individuos para entrenamiento
        train_data.append(individual_data)
        train_labels.append(jaime_labels[i])
    else:  # Últimos 2 individuos para validación
        validation_data.append(individual_data)
        validation_labels.append(jaime_labels[i])

# Convertir listas a arrays de NumPy
train_data = np.concatenate(train_data, axis=0)
train_labels = to_categorical(train_labels, num_classes=3)
validation_data = np.concatenate(validation_data, axis=0)
validation_labels = to_categorical(validation_labels, num_classes=3)

model = Sequential([
    LSTM(50, input_shape=(train_data.shape[1], train_data.shape[2])),  # 50 unidades LSTM
    Dense(3, activation='softmax')  # Capa de salida para 3 clases
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_labels, epochs=10, validation_data=(validation_data, validation_labels))


Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 1.1411 - accuracy: 0.5000 - val_loss: 1.9425 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 42ms/step - loss: 1.0058 - accuracy: 0.6667 - val_loss: 1.9742 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 43ms/step - loss: 0.9127 - accuracy: 0.8333 - val_loss: 1.9884 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 40ms/step - loss: 0.8371 - accuracy: 0.8333 - val_loss: 1.9969 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 38ms/step - loss: 0.7700 - accuracy: 0.8333 - val_loss: 2.0064 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 38ms/step - loss: 0.7087 - accuracy: 0.8333 - val_loss: 2.0218 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 45ms/step - loss: 0.6517 - accuracy: 0.8333 - val_loss: 2.0429 - val_accuracy: 0.0000e+00
E

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

# Assuming jaime_data is your full dataset and jaime_labels are the corresponding labels

# Prepare full training data and labels
full_train_data = []
full_train_labels = []

for i, (key, df) in enumerate(train_data.items()):
    # Convert each DataFrame to a 3D array (samples, time steps, features)
    individual_data = np.expand_dims(df.values, axis=0)
    full_train_data.append(individual_data)
    full_train_labels.append(jaime_labels[i])

# Convert lists to numpy arrays
full_train_data = np.concatenate(full_train_data, axis=0)
full_train_labels = np.array(full_train_labels)

# Convert labels to categorical (one-hot encoding)
num_classes = len(np.unique(full_train_labels))  # assuming this is how many unique labels you have
full_train_labels = to_categorical(full_train_labels, num_classes=num_classes)

# Define your LSTM model
model = Sequential([
    LSTM(30, input_shape=(full_train_data.shape[1], full_train_data.shape[2])),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the full data
history = model.fit(full_train_data, full_train_labels, epochs=10)


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 1.1040 - accuracy: 0.4286
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 1.0124 - accuracy: 0.4286
Epoch 3/10
1/1 [==============================] - 0s 15ms/step - loss: 0.9258 - accuracy: 0.5714
Epoch 4/10
1/1 [==============================] - 0s 14ms/step - loss: 0.8447 - accuracy: 0.7143
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.7724 - accuracy: 0.8571
Epoch 6/10
1/1 [==============================] - 0s 14ms/step - loss: 0.7080 - accuracy: 0.8571
Epoch 7/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6503 - accuracy: 0.8571
Epoch 8/10
1/1 [==============================] - 0s 14ms/step - loss: 0.5997 - accuracy: 0.8571
Epoch 9/10
1/1 [==============================] - 0s 18ms/step - loss: 0.5596 - accuracy: 0.8571
Epoch 10/10
1/1 [==============================] - 0s 20ms/step - loss: 0.5279 - accuracy: 1.0000


In [ ]:
full_train_data.shape

(7, 142, 28)

In [ ]:
import os
from PIL import Image
import numpy as np
import re

# base_path = '/content/drive/MyDrive/Fish2'
base_path = 'Fish2'

# Function to preprocess an image
def preprocess_image(image_path, target_size=(224, 224)):
    with Image.open(image_path) as img:
        img = img.convert('RGB')  # Convert image to RGB
        img = img.resize(target_size)
        image_array = np.array(img)
        image_array = image_array / 255.0
    return image_array

# Function to create sequences with proper padding for intermittent visibility
def create_custom_padded_sequences(base_path, appearance_times, max_sequence_length):
    sequences = {}  # Dictionary to hold image sequences for each individual

    # Sort the folder names numerically
    sorted_folders = sorted(os.listdir(base_path), key=lambda x: int(re.search(r'\d+', x).group()))

    # Iterate over the sorted folders and the images within them
    for folder_name in sorted_folders:
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            # List to hold preprocessed images for the current folder
            images = [preprocess_image(os.path.join(folder_path, img_name))
                      for img_name in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group()))]
            sequences[folder_name] = images

    # Apply custom padding based on appearance times
    padded_sequences = []
    for folder_name, images in sequences.items():
        sequence_padding = []
        image_index = 0  # Index to keep track of the current image
        intervals = appearance_times.get(folder_name, [(0, max_sequence_length)])

        for frame_number in range(max_sequence_length):
            # Check if the frame number is within any visibility interval
            if any(start <= frame_number < end for start, end in intervals):
                if image_index < len(images):  # Check if there are still images left to append
                    sequence_padding.append(images[image_index])  # add the next image
                    image_index += 1
                else:
                    # If no images are left, append a blank image
                    sequence_padding.append(np.zeros((224, 224, 3)))
            else:
                # If not visible, append a blank image
                sequence_padding.append(np.zeros((224, 224, 3)))
        padded_sequences.append(sequence_padding)


    return padded_sequences

# Define appearance times including all visible intervals
# Define appearance times including all visible intervals
appearance_times = {
    '0' : [(0,136)],
    '1': (0, 139),
    '2-21-31-132': [(0, 21), (31, 132)],
    '3': (0, 141),
    '4-060': (60, 141),
    '5-117': (117, 141),
    '6-122': [(122, 141)]
}

max_sequence_length = 141  # Or the length of your video sequence
padded_image_sequences = create_custom_padded_sequences(base_path, appearance_times, max_sequence_length)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv2D, MaxPooling2D, Flatten, Masking


def create_cnn_model():
    # Load ResNet50 with pre-trained weights, without the top classification layer
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the layers of the base_model
    for layer in base_model.layers:
        layer.trainable = False

    # Add a Flatten layer or GlobalAveragePooling layer here depending on your needs
    x = base_model.output
    x = Flatten()(x)  # or you can use GlobalAveragePooling2D()

    # Create the full model
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Define LSTM model that will take the CNN features as input
def create_lstm_model():
    cnn_model = create_cnn_model()
    model = Sequential()
    # model.add(Masking(mask_value=0., input_shape=(141, 224, 224, 3)))
    model.add(TimeDistributed(cnn_model, input_shape=(max_sequence_length, 224, 224, 3)))
    model.add(LSTM(units=50, return_sequences=False))
    # add more layers as needed...
    model.add(Dense(units=3, activation='softmax'))
    return model


In [ ]:
from tensorflow.keras.optimizers import Adam
jaime_labels = [0, 0, 2, 2, 1, 2, 1]

# Assume padded_image_sequences is a list of NumPy arrays (your image data)
# and jaime_labels is a list or array of labels
class AdamW(Adam):
    def __init__(self, weight_decay, **kwargs):
        super(AdamW, self).__init__(**kwargs)
        self.weight_decay = weight_decay

    def _resource_apply_dense(self, grad, var, apply_state=None):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)  # handle learning rate decay
        wd = self.weight_decay * lr_t
        var.assign_sub(wd * var, use_locking=self._use_locking)
        return super(AdamW, self)._resource_apply_dense(grad, var, apply_state)

# Convert the list of labels to a numpy array if they aren't already
jaime_labels_array = np.array(jaime_labels)

# If padded_image_sequences is a list of arrays, convert it to a single NumPy array
padded_image_sequences_array = np.stack(padded_image_sequences)
print(padded_image_sequences_array.shape)  # Should output something like (num_samples, sequence_length, height, width, channels)
print(jaime_labels_array.shape)  # Should output (num_samples,)

# Now padded_image_sequences_array and jaime_labels_array are your full dataset and labels

# Initialize AdamW optimizer with weight decay
adam_optimizer_custom = AdamW(weight_decay=0.01, learning_rate=0.0001)

# Compile the hybrid CNN-LSTM model
hybrid_model = create_lstm_model()
hybrid_model.compile(optimizer=adam_optimizer_custom, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model using the full dataset
hybrid_model.fit(
    padded_image_sequences_array,
    jaime_labels_array,
    batch_size = 6 ,
    epochs=10  # or however many epochs you wish to train for
)


(7, 141, 224, 224, 3)
(7,)
Epoch 1/10
2/2 [==============================] - 53s 7s/step - loss: 1.1000 - accuracy: 0.2857
Epoch 2/10
2/2 [==============================] - 3s 447ms/step - loss: 1.6718 - accuracy: 0.4286
Epoch 3/10
2/2 [==============================] - 3s 462ms/step - loss: 1.4235 - accuracy: 0.4286
Epoch 4/10
2/2 [==============================] - 3s 449ms/step - loss: 1.2465 - accuracy: 0.4286
Epoch 5/10
2/2 [==============================] - 3s 449ms/step - loss: 1.2260 - accuracy: 0.4286
Epoch 6/10
2/2 [==============================] - 3s 452ms/step - loss: 1.1968 - accuracy: 0.4286
Epoch 7/10
2/2 [==============================] - 3s 464ms/step - loss: 1.1548 - accuracy: 0.4286
Epoch 8/10
2/2 [==============================] - 3s 442ms/step - loss: 1.1316 - accuracy: 0.4286
Epoch 9/10
2/2 [==============================] - 3s 426ms/step - loss: 1.1029 - accuracy: 0.4286
Epoch 10/10
2/2 [==============================] - 3s 432ms/step - loss: 1.0837 - accuracy: 0

# Test ensemble learning

preprocessing test data

In [ ]:
import os
from PIL import Image
import numpy as np
import re

# base_path = '/content/drive/MyDrive/Fish'
base_path = 'Fish'

# Function to preprocess an image
def preprocess_image(image_path, target_size=(224, 224)):
    with Image.open(image_path) as img:
        img = img.convert('RGB')  # Convert image to RGB
        img = img.resize(target_size)
        image_array = np.array(img)
        image_array = image_array / 255.0
    return image_array

# Function to create sequences with proper padding for intermittent visibility
def create_custom_padded_sequences(base_path, appearance_times, max_sequence_length):
    sequences = {}  # Dictionary to hold image sequences for each individual

    # Sort the folder names numerically
    sorted_folders = sorted(os.listdir(base_path), key=lambda x: int(re.search(r'\d+', x).group()))

    # Iterate over the sorted folders and the images within them
    for folder_name in sorted_folders:
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            # List to hold preprocessed images for the current folder
            images = [preprocess_image(os.path.join(folder_path, img_name))
                      for img_name in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group()))]
            sequences[folder_name] = images

    # Apply custom padding based on appearance times
    padded_sequences = []
    for folder_name, images in sequences.items():
        print(folder_name)
        sequence_padding = []
        image_index = 0  # Index to keep track of the current image
        intervals = appearance_times.get(folder_name, [(0, max_sequence_length)])
        print(f"Folder: {folder_name}, Intervals: {intervals}")  # Add this line to check the intervals format

        for frame_number in range(max_sequence_length):
            # Check if the frame number is within any visibility interval
            if any(start <= frame_number < end for start, end in intervals):
                if image_index < len(images):  # Check if there are still images left to append
                    sequence_padding.append(images[image_index])  # add the next image
                    image_index += 1
                else:
                    # If no images are left, append a blank image
                    sequence_padding.append(np.zeros((224, 224, 3)))
            else:
                # If not visible, append a blank image
                sequence_padding.append(np.zeros((224, 224, 3)))
        padded_sequences.append(sequence_padding)


    return padded_sequences

# Define appearance times including all visible intervals
# Define appearance times including all visible intervals
appearance_times = {
    '0' :[(0,57)],
    '1': [(0, 57)],
    '2': [(0, 57)],
    '3': [(0, 57)],
    '4': [(0, 37)],
    '5': [(0, 57)],
    '6': [(0, 57)],
    '22':[(24, 57)],
    '25': [(27, 57)],
    '40': [(40, 57)]
}

max_sequence_length = 141  # Or the length of your video sequence
X_test_images = create_custom_padded_sequences(base_path, appearance_times, max_sequence_length)


0
Folder: 0, Intervals: [(0, 57)]
1
Folder: 1, Intervals: [(0, 57)]
2
Folder: 2, Intervals: [(0, 57)]
3
Folder: 3, Intervals: [(0, 57)]
4
Folder: 4, Intervals: [(0, 37)]
5
Folder: 5, Intervals: [(0, 57)]
6
Folder: 6, Intervals: [(0, 57)]
22
Folder: 22, Intervals: [(24, 57)]
25
Folder: 25, Intervals: [(27, 57)]
40
Folder: 40, Intervals: [(40, 57)]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# test_data_numeric = pd.read_csv('/content/drive/MyDrive/CollectedData_katia.csv', skiprows=3)
test_data_numeric = pd.read_csv('TrainDataFishVideo.csv', skiprows=3)

print(test_data_numeric.shape)

test_data = prepare_individuals(test_data_numeric, target_length=56, body_parts=body_parts, num_individuals=10)


# Prepare test data with padding
test_data_padded = []

for key, df in test_data.items():
    # Convert the DataFrame to a 3D array (samples, time steps, features)
    individual_data = np.expand_dims(df.values, axis=0)
    # Pad sequences to match the model's expected input shape (142 time steps)
    individual_data_padded = pad_sequences(individual_data, maxlen=142, dtype='float32', padding='post', truncating='post', value=0.0)
    test_data_padded.append(individual_data_padded)

# Convert list to a NumPy array
test_data_padded = np.concatenate(test_data_padded, axis=0)




(56, 311)


In [ ]:
test_data_padded.shape

(10, 142, 28)

In [ ]:
def ensemble_predictions(model1, model2, data1, data2, weights=[0.5, 0.5]):
    # Ensure weights sum to 1
    weights = np.array(weights) / sum(weights)

    # Predict with each model
    preds1 = model1.predict(data1)  # Assuming this returns class probabilities for the numerical data model
    print(preds1)
    print(preds1.shape)
    X_test_array = np.stack(data2)
    batch_size = 1  # Adjust the batch size to fit your system's memory
    num_samples = X_test_array.shape[0]

    preds2 = []
    for i in range(0, X_test_array.shape[0], batch_size):
        end_index = i + batch_size
        # Ensure we don't go past the end of the array on the last batch
        if end_index > X_test_array.shape[0]:
            end_index = X_test_array.shape[0]
        batch_predictions = hybrid_model.predict(X_test_array[i:end_index])
        preds2.append(batch_predictions)

    # Concatenate all batch predictions into a single array
    preds2 = np.vstack(preds2)  # Use vstack to stack arrays vertically
    print(preds2)
    print(preds2.shape)


    # Combine predictions
    weights = np.array(weights)

    combined_preds = (weights[0] * preds1) + (weights[1] * preds2)

    # Final prediction
    final_preds = np.argmax(combined_preds, axis=1)

    return final_preds
ensemble_predictions(model, hybrid_model,test_data_padded , X_test_images)

1/1 [==============================] - 0s 376ms/step
[[0.3258123  0.3230889  0.35109872]
 [0.32581228 0.32308882 0.35109887]
 [0.32581225 0.3230888  0.3510989 ]
 [0.32581234 0.323089   0.35109863]
 [0.32581225 0.32308882 0.35109892]
 [0.32581237 0.32308894 0.35109866]
 [0.32581225 0.32308877 0.35109898]
 [0.3258124  0.32308927 0.35109836]
 [0.32581228 0.3230891  0.3510986 ]
 [0.3258123  0.32308897 0.3510987 ]]
(10, 3)
1/1 [==============================] - 0s 109ms/step
[[0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]]
(10, 3)


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
# true_labels = [0, 1, 0, 2, 1, 3, 3,1,1]  # Assuming you have a corresponding y_test


In [ ]:
from sklearn.metrics import accuracy_score

# Assuming you have a variable `true_labels` which contains the true class indices
true_labels = np.array([0, 1, 0, 2, 1, 3, 3,1,1,3])

# Let's say `padded_image_sequences_array` is your test data
# You would get predictions from your model like this:
predicted_classes = ensemble_predictions(model, hybrid_model,test_data_padded , X_test_images)

# Now you have the predicted class indices, you can compare them with the true labels
accuracy = accuracy_score(true_labels, predicted_classes)

# Print out the accuracy
print(f"Model accuracy: {accuracy * 100:.2f}%")


1/1 [==============================] - 0s 197ms/step
[[0.3258123  0.3230889  0.35109872]
 [0.32581228 0.32308882 0.35109887]
 [0.32581225 0.3230888  0.3510989 ]
 [0.32581234 0.323089   0.35109863]
 [0.32581225 0.32308882 0.35109892]
 [0.32581237 0.32308894 0.35109866]
 [0.32581225 0.32308877 0.35109898]
 [0.3258124  0.32308927 0.35109836]
 [0.32581228 0.3230891  0.3510986 ]
 [0.3258123  0.32308897 0.3510987 ]]
(10, 3)
1/1 [==============================] - 0s 137ms/step
[[0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]
 [0.2806388  0.32924953 0.3901117 ]]
(10, 3)
Model accuracy: 10.00%
